In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

In [165]:
countries = [
    "Austria",
    "Belgium",
    "Canada",
    "Denmark",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Ireland",
    "Italy",
    "Japan",
    "Netherlands",
    "Portugal",
    "Spain",
    "Sweden",
    "United Kingdom",
    "United States"
]

In [166]:
cpi = pd.read_csv ('PRICES_CPI_19062024161712242.csv')
cpi = cpi[cpi['Country'].isin(countries)]
cpi = cpi[(cpi['Subject'] == 'CPI: 01-12 - All items')]
cpi = cpi[['Country', 'TIME', 'Subject', 'Unit', 'MEASURE', 'Value']]
cpi = cpi.rename (columns={'Value': 'cpi_value'})
          
private_consumption = pd.read_csv ('EO_19062024154150215.csv')
private_consumption = private_consumption[private_consumption['Country'].isin(countries)]
private_consumption = private_consumption[['Country', 'VARIABLE', 'TIME', 'Value']]
private_consumption = private_consumption.rename (columns={'Value': 'consumption_value'})

exchange_rates = pd.read_csv ('SNA_TABLE4_19062024161942742.csv')
exchange_rates = exchange_rates[exchange_rates['Country'].isin(countries)]
exchange_rates = exchange_rates[(exchange_rates['Transaction'] == 'Exchange rates, period-average')]
exchange_rates = exchange_rates[['Country', 'TIME', 'Value']]
exchange_rates = exchange_rates.rename (columns={'Value': 'rate_value'})

exchange_rates

,Country,TIME,rate_value
32,Spain,1970,0.420708
33,Spain,1971,0.417515
34,Spain,1972,0.386279
35,Spain,1973,0.350150
36,Spain,1974,0.346703
...,...,...,...
4910,Austria,1997,0.886917
4911,Austria,1998,0.899622
4912,Austria,1999,0.938283
4913,Austria,2000,1.082705


In [167]:
cpi = cpi[(cpi['Unit'] == 'Index') & (cpi['MEASURE'] == 'IXOB')]

In [175]:
merged1 = pd.merge(cpi, exchange_rates, on = ['Country', 'TIME'], how = 'inner').drop_duplicates()
final = pd.merge(merged1, private_consumption, on = ['Country', 'TIME'], how = 'inner')
final = final[['Country', 'TIME', 'cpi_value', 'rate_value', 'consumption_value']]
final['consumption_in_usd'] = (final['consumption_value'].apply(lambda x: float(x))) / (final['rate_value'].apply(lambda x: float(x)))
final

,Country,TIME,cpi_value,rate_value,consumption_value,consumption_in_usd
0,Netherlands,1970,22.96813,1.642684,1.359985e+11,8.279044e+10
1,Netherlands,1971,24.68562,1.595982,1.404890e+11,8.802666e+10
2,Netherlands,1972,26.61179,1.456408,1.454079e+11,9.984011e+10
3,Netherlands,1973,28.74664,1.268565,1.512213e+11,1.192066e+11
4,Netherlands,1974,31.50385,1.219935,1.568194e+11,1.285473e+11
...,...,...,...,...,...,...
473,Portugal,1997,67.95118,0.874455,9.856798e+10,1.127193e+11
474,Portugal,1998,69.69939,0.898357,1.032843e+11,1.149702e+11
475,Portugal,1999,71.33043,0.938283,1.087225e+11,1.158739e+11
476,Portugal,2000,73.36551,1.082705,1.127315e+11,1.041202e+11


In [176]:
us_info = final[final['Country'] == 'United States'][['TIME', 'cpi_value', 'consumption_value']]
us_info = us_info.rename(columns={'cpi_value': 'us_cpi_value', 'consumption_value': 'us_consumption_value'})

final2 = pd.merge(final, us_info, on = 'TIME')
final2

,Country,TIME,cpi_value,rate_value,consumption_value,consumption_in_usd,us_cpi_value,us_consumption_value
0,Netherlands,1970,22.968130,1.642684,1.359985e+11,8.279044e+10,16.38068,3.271963e+12
1,United Kingdom,1970,8.412123,0.416667,3.932922e+11,9.439005e+11,16.38068,3.271963e+12
2,Belgium,1970,19.292080,1.239468,8.245409e+10,6.652377e+10,16.38068,3.271963e+12
3,Sweden,1970,13.213290,5.173210,1.004312e+12,1.941370e+11,16.38068,3.271963e+12
4,France,1970,14.606270,0.846731,4.360730e+11,5.150077e+11,16.38068,3.271963e+12
...,...,...,...,...,...,...,...,...
473,Denmark,2001,78.025000,8.322817,7.543000e+11,9.063037e+10,74.70631,9.393886e+12
474,Spain,2001,73.842780,1.116533,5.565560e+11,4.984680e+11,74.70631,9.393886e+12
475,Italy,2001,77.147870,1.116533,1.009184e+12,9.038552e+11,74.70631,9.393886e+12
476,Canada,2001,77.258360,1.548840,7.854950e+11,5.071505e+11,74.70631,9.393886e+12


In [179]:
final2['real_exchange_rate'] = (final2['rate_value'] * final2['us_cpi_value']) / final2['cpi_value']
final2['relative_consumption'] = final2['consumption_in_usd'] / final2['us_consumption_value']
final2[final2['Country'] == 'United Kingdom']

,Country,TIME,cpi_value,rate_value,consumption_value,consumption_in_usd,us_cpi_value,us_consumption_value,real_exchange_rate,relative_consumption
1,United Kingdom,1970,8.412123,0.416667,3.932922e+11,9.439005e+11,16.38068,3.271963e+12,0.811363,0.288481
15,United Kingdom,1971,9.206635,0.410920,4.068733e+11,9.901522e+11,17.08387,3.396916e+12,0.762505,0.291486
29,United Kingdom,1972,9.857644,0.400390,4.334743e+11,1.082630e+12,17.64290,3.604970e+12,0.716605,0.300316
43,United Kingdom,1973,10.764160,0.408171,4.587787e+11,1.123987e+12,18.73283,3.783374e+12,0.710339,0.297086
57,United Kingdom,1974,12.491160,0.427756,4.534879e+11,1.060156e+12,20.80371,3.751681e+12,0.712417,0.282581
71,United Kingdom,1975,15.514930,0.452041,4.524082e+11,1.000812e+12,22.70583,3.836749e+12,0.661554,0.260849
85,United Kingdom,1976,18.084130,0.556510,4.530504e+11,8.140920e+11,24.01023,4.050586e+12,0.738876,0.200981
99,United Kingdom,1977,20.948700,0.573272,4.504871e+11,7.858173e+11,25.57130,4.221781e+12,0.699772,0.186134
113,United Kingdom,1978,22.679720,0.521505,4.739781e+11,9.088659e+11,27.52264,4.406546e+12,0.632865,0.206254
127,United Kingdom,1979,25.723630,0.472181,4.970612e+11,1.052692e+12,30.62017,4.511288e+12,0.562062,0.233346


In [178]:
results = []
for country, group in final2.groupby('Country'):
    corr, _ = pearsonr(group['real_exchange_rate'], group['relative_consumption'])
    results.append({'Country': country, 'Pearson Correlation': corr})

# Create a DataFrame from results
correlation_df = pd.DataFrame(results)

print(correlation_df)

           Country  Pearson Correlation
0          Austria            -0.917998
1          Belgium            -0.925521
2           Canada            -0.893086
3          Denmark            -0.474982
4          Finland            -0.590104
5           France            -0.690417
6          Germany            -0.961911
7           Greece            -0.967622
8          Ireland            -0.868225
9            Italy            -0.214162
10           Japan            -0.874359
11     Netherlands            -0.935660
12        Portugal            -0.030176
13           Spain             0.031305
14          Sweden            -0.648106
15  United Kingdom             0.047832
16   United States                  NaN


c:\Anaconda3-202307\Lib\site-packages\scipy\stats\_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [ ]:
#Detrend data
#Deseasonalise data - maybe not because annual
